In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth, dayofweek, avg, count, lit, stddev
from pyspark.sql.window import Window

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from datetime import datetime, timedelta

import json
from pyspark.ml.regression import GBTRegressionModel

bronze_path = "Files/bronze/transacciones_servicios/"
silver_path = "Files/silver/transacciones_servicios/"
gold_path = "Files/gold/dataset_recurrencia_servicios/"

modelo_fecha_path = "Files/models/modelo_servicios_fecha/"
modelo_monto_path = "Files/models/modelo_servicios_monto/"

notebook_version = "1.0"

print(f"Ruta Bronze: {bronze_path}")
print(f"Ruta Silver: {silver_path}")
print(f"Ruta Gold: {gold_path}")

StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 3, Finished, Available, Finished)

Ruta Bronze: Files/bronze/transacciones_servicios/
Ruta Silver: Files/silver/transacciones_servicios/
Ruta Gold: Files/gold/dataset_recurrencia_servicios/


In [4]:
modelo_fecha = GBTRegressionModel.load("Files/models/modelo_servicios_fecha")
modelo_monto = GBTRegressionModel.load("Files/models/modelo_servicios_monto")

StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 6, Finished, Available, Finished)

In [5]:
feature_cols = [
    "dias_diferencia_anterior_transaccion",
    "monto",
    "dia_transaccion",
    "dia_semana",
    "mes_transaccion",
    "conteo_transacciones_hist",
    "promedio_dias_hist"
]

def predecir_transaccion(cuenta_origen, cuenta_destino):

    df_input = spark.table("fs_servicios") \
        .filter((col("cedula_remitente") == cuenta_origen) &
                (col("numero_cuenta_servicio") == cuenta_destino)) \
        .orderBy(col("fecha_transaccion").desc()) \
        .limit(1)

    if df_input.count() == 0:
        return json.dumps({
            "error": "No se encontraron transacciones para esa relación cuenta_origen-cuenta_destino"
        })

    ultima_fecha_str = df_input.select("fecha_transaccion").collect()[0][0]
    ultima_fecha = datetime.strptime(ultima_fecha_str, "%Y-%m-%d")

    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    df_input = assembler.transform(df_input)

    pred_fecha = modelo_fecha.transform(df_input).select("prediction").collect()[0][0]

    pred_monto = modelo_monto.transform(df_input).select("prediction").collect()[0][0]

    fecha_estimacion = ultima_fecha + timedelta(days=int(round(pred_fecha)))

    resultado_json = json.dumps({
        "cedula_remitente": cuenta_origen,
        "numero_cuenta_servicio": cuenta_destino,
        "fecha_estimada": fecha_estimacion.strftime("%Y-%m-%d"),
        "monto_estimado": round(pred_monto, 0)
    })

    mensaje = (
        f"✨ Te recomendamos realizar tu próxima transacción "
        f"el **{fecha_estimacion.strftime('%Y-%m-%d')}** por un valor aproximado de **${pred_monto}**. "
        f"💪 ¡Confía en tu progreso y mantén el impulso!"
    )

    return resultado_json, mensaje

StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 7, Finished, Available, Finished)

In [7]:
print(predecir_transaccion("00104916838", "102490724"))


StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 9, Finished, Available, Finished)

('{"cedula_remitente": "00104916838", "numero_cuenta_servicio": "102490724", "fecha_estimada": "2025-09-30", "monto_estimado": 4867.0}', '✨ Te recomendamos realizar tu próxima transacción el **2025-09-30** por un valor aproximado de **$4867.316032609032**. 💪 ¡Confía en tu progreso y mantén el impulso!')


In [8]:
print(predecir_transaccion("00117595462", "2519910104")[1])


StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 10, Finished, Available, Finished)

✨ Te recomendamos realizar tu próxima transacción el **2025-09-05** por un valor aproximado de **$574.0013103868091**. 💪 ¡Confía en tu progreso y mantén el impulso!


In [6]:
display(spark.sql("select * from fs_servicios"))

StatementMeta(, 46242c82-082d-4293-a711-e78cf24a1ed7, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 724d2cf6-d7b5-42dc-855a-fcc5defdd740)